## Trends in music sales - Performance Optimization
In this  project, I will use my SQL skills to answer business questions for the Chinook record store.

The database is provided as an SQLite database file called chinook.db. A schematic overview of the database tables and content can be found here: https://s3.amazonaws.com/dq-content/190/chinook-schema.svg. An overview of the database tables is also given below.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Context and initial BI questions
The Chinook record store has just signed a deal with a new record label, and I have been tasked with selecting the first three albums that will be added to the store.

We have the artist names, and the genre of music they produce:

Artist Name |	Genre   
Regal |	Hip-Hop   
Red Tone |	Punk   
Meteor and the Girls |	Pop   
Slim Jim Bites |	Blues   

I need to:
1. Identify which genres sell the most tracks in the USA.
2. Write up a summary of my findings
3. Make a recommendation for the three artists whose albums we should purchase for the store.

In [3]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.*
    FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    SUM(uts.quantity) tracks_sold, -- Using SUM to include multiple sales pr invoice
    ROUND(100 * CAST(SUM(uts.quantity) AS FLOAT) / (
        SELECT SUM(quantity) FROM usa_tracks_sold
    ), 2) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t ON t.track_id = uts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


## Recommendation for sale campaign
The initial analysis shows that the following relevant genres are the best selling in USA:

Alternative & Punk	12.37%   
Blues 3.43%     
Pop	2.09%   

It is therefore recommended that the campaign should focus on the following artists: Red Tone (Punk), Meteor and the Girls (Pop) and Slim Jim Bites (Blues).

Also note that these genres only make up 18% of the total sales. It could therefore be of interest to watch for artists and albums from the 'rock' genre, which accounts for 53% of the total sales.

## Identifiying the best performing support agents
Next management asks me to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing better and to identify supporting relevant information to the sales.

In [4]:
%%sql
/* I will make a table with 'Top Sales by support agent', 'Total Sales', 'Top Genre sold by support agent' */

WITH rep_sales AS
   (
    SELECT
       c.customer_id,
       c.support_rep_id,
       i.invoice_id,
       SUM(i.total) total
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1
   ),
    rep_top_genre AS
    (
    SELECT
        g.name,
        il.invoice_id
    FROM genre g
    INNER JOIN track t ON t.genre_id = g.genre_id
    INNER JOIN invoice_line il ON il.track_id = t.track_id
    GROUP BY 2
    )

SELECT
    e.first_name || " " || e.last_name employee,
    ROUND(SUM(rs.total),1) total_sales,
    rtg.name employee_sale_top_genre
    
FROM rep_sales rs
INNER JOIN employee e ON e.employee_id = rs.support_rep_id
INNER JOIN rep_top_genre rtg ON rtg.invoice_id = rs.invoice_id
GROUP BY 1;

Done.


employee,total_sales,employee_sale_top_genre
Jane Peacock,1731.5,Metal
Margaret Park,1584.0,Easy Listening
Steve Johnson,1393.9,Latin


## Observations and a possible opportunity to increase total sales
When commenting on the results above it is relevant to compare with the top genres sold overall. Comparing to the first table we initially note that:

Genre | Percentage of total sale
Metal | 11.8%   
Easy Listening | 1.24%   
Latin | 2.09%

This leads to the following observation and a possible opportunity to increase future sales:
Each employee seems to have a niche where they perform a major part of their sale.

It is noteworthy that the second highest contributor to the total sale has a very niche top genre. This could indicate a potential to increase sales for other niches further by hiring support agents with a keen knowledge about other music genres.

## Analysis of global sales
Next the stakeholder asks me to calculate data, for each country, on the:

* total number of customers   
* total value of sales   
* average value of sales per customer   
* average order value

To perform this analysis in a meaningful way, it is relevant to create an output table where countries with minor sales are grouped into a combined category called 'Other'.

In [5]:
%%sql

WITH country_or_other AS
    (
     SELECT
        -- I sort countries with single sales into a group called 'Other' ELSE the country is included 
        CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
        END AS country,
        c.customer_id,
        i.total, --included for testing purposes, is not used in final output
        il.*
        FROM customer c
        INNER JOIN invoice i ON c.customer_id = i.customer_id
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    )

SELECT
    country,
    COUNT(DISTINCT customer_id) customers,
    ROUND(SUM(unit_price), 2) total_sales,
    ROUND(SUM(unit_price) / COUNT(DISTINCT customer_id), 2) avg_value_pr_customer,
    ROUND(SUM(unit_price) / COUNT(DISTINCT invoice_id), 2) average_order
FROM country_or_other
GROUP BY country
ORDER BY total_sales DESC

Done.


country,customers,total_sales,avg_value_pr_customer,average_order
Other,15,1094.94,73.0,7.45
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72


## Opportunities in the global market
Based on the data, there may be opportunity in the following European countries:

* Czech Republic   
* United Kingdom   
* Portugal

Keep in mind that because the amount of data from each of these countries is very low, these observations can only be used as a first indicator. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us a high confidence.

A good approach would be to run small campaigns in these countries, collecting and analyzing the purchases of new customers to make sure that these trends are confirmed.